### 1 . Save index in a directory

In [ ]:

# Check from the latest documentation : https://docs.llamaindex.ai/en/stable/examples/vector_stores/FaissIndexDemo/

from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import faiss

# 1. Load your documents
documents = SimpleDirectoryReader(input_dir="../data/docs/").load_data()


# 2. Initialize embeddings, used for encoding documents into embedding
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


# 3. Create FAISS index
dimension = 384  # Dimension for MiniLM embeddings
faiss_index = faiss.IndexFlatL2(dimension)

# 4. Construct the FaissVectorStore
faiss_vector_store = FaissVectorStore(faiss_index=faiss_index)


# 5. Create a StorageContext to use with LlamaIndex
storage_context = StorageContext.from_defaults(vector_store=faiss_vector_store)

# 6. Build index
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, embed_model=embed_model)
retriever = index.as_retriever()


# 7. Save index to disk
index.storage_context.persist("storage_dir")


/home/nausheenfatma/.cache/pypoetry/virtualenvs/canadaimmigrationhelpdesk-usinggenai-s0qUKeX3-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Load index from directory

In [2]:

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

from llama_index.core import StorageContext, load_index_from_storage
from llama_index.vector_stores.faiss import FaissVectorStore

from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama2", request_timeout=100.0)

# Restore settings
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage_dir")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage_dir"
)
index = load_index_from_storage(storage_context=storage_context)

In [3]:
retriever = index.as_retriever()

In [ ]:

# Query function
def query_rag_system(question):
    retrieved_docs = retriever.retrieve(question)
    context = "\n".join([doc.text for doc in retrieved_docs])    
    prompt = f"Context:\n{context}\n\n Question: {question}\n\n\n\nAnswer:"
    response = llm.complete(prompt)
    return response

def main():
    ### Example
    question = "Can I apply for the Rural and Northern Immigration Pilot now?"
    answer = query_rag_system(question=question)
    print (answer)

In [5]:
question = "Can I apply for the Rural and Northern Immigration Pilot now?"

In [6]:
answer = query_rag_system(question=question)

In [7]:
answer

CompletionResponse(text='Sorry, but as of August 31, 2024, the Rural and Northern Immigration Pilot has ended. Therefore, you cannot apply for the pilot program at this time. However, you may be eligible to apply for a work permit if you have an eligible job offer in Canada. You can also continue to process applications received before the end of the pilot on August 31, 2024.\n\nIf you are interested in immigrating to Canada through other programs, you can explore the options available on the Immigration, Refugees and Citizenship Canada website. Additionally, you may want to consider consulting with a registered immigration lawyer or consultant to discuss your eligibility and options for immigrating to Canada.', additional_kwargs={'tool_calls': []}, raw={'model': 'llama2', 'created_at': '2025-04-10T19:14:38.750931835Z', 'done': True, 'done_reason': 'stop', 'total_duration': 19365050449, 'load_duration': 12953616, 'prompt_eval_count': 1353, 'prompt_eval_duration': 125778683, 'eval_count